In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Иницијализација на Web Driver за Selenium
driver = webdriver.Chrome()
driver.get("https://kosuli.mk")
wait = WebDriverWait(driver, 10)

# Дефинирање на речник, dictionary на категориите кои ги скрејпам со големи букви
menu_targets = {
    "Брендови": {"CASA MODA", "FF-FASHION", "VENTI"},
    "Кратки ракави": {"МОДЕРН ФИТ"},
}

# Празно dictionary за чување на УРЛ's
extracted_urls = {}

for top_menu, targets in menu_targets.items():
    # Се чека елементот да биде пронајден со hover 
    link = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//a[contains(normalize-space(), '{top_menu}')]")
    ))
    actions = ActionChains(driver)
    actions.move_to_element(link).perform()

    # Се чеза за елементот да биде видлив за да може да се лоцира
    submenu = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//li[a[contains(normalize-space(), '{top_menu}')]]/ul")
    ))

    # Се зачувуваат сите линкови од секоја категорија 
    submenu_anchors = submenu.find_elements(By.TAG_NAME, "a")

    for a in submenu_anchors:
        text = a.text.strip().upper()
        href = a.get_attribute("href")
        if text in targets and href:
            # Се зачувува во варијабла линкот 
            key = text.lower().replace(" ", "_")
            extracted_urls[key] = href

# Затварање на бровсерот односно исклучување на селениум
driver.quit()

# Зачувување на варијабли, главната варијбала страницата и потоа секое урл посебно
base_url = "https://kosuli.mk"
url_casa_moda = extracted_urls.get("casa_moda")
url_ff_fashion = extracted_urls.get("ff-fashion")
url_venti = extracted_urls.get("venti")
url_modern_fit = extracted_urls.get("модерн_фит")

# Принтање на секој линк
print("Casa Moda :", url_casa_moda)
print("FF-Fashion:", url_ff_fashion)
print("Venti     :", url_venti)
print("Modern Fit:", url_modern_fit)


Casa Moda : https://kosuli.mk/product-category/casa-moda/
FF-Fashion: https://kosuli.mk/product-category/f-fashion/
Venti     : https://kosuli.mk/product-category/venti/
Modern Fit: https://kosuli.mk/product-category/modern-fit/


In [4]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

# Сетирање на селениум веб драјверот и исклучување на сликите за побрзо лоадање
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

product_urls = []
page = 1

# Изминување на сите страници за собирање на урлта на продуктите 
while True:
    driver.get(f"{url_venti}page/{page}/")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR, "ul.products li.product a.woocommerce-LoopProduct-link")
    if not links:
        break

    for a in links:
        href = a.get_attribute("href")
        if href and href not in product_urls:
            product_urls.append(href)

    try:
        driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
        page += 1
    except NoSuchElementException:
        break

print(f"→ Found {len(product_urls)} products across {page} pages.")

# Скрејпање на деталите за секој продукт
data = []
for idx, href in enumerate(product_urls, start=1):
    driver.get(href)
    time.sleep(1)

    # Скрејпрање на името
    name = driver.find_element(By.TAG_NAME, "h1").text.strip()

    # Скрејпање на цената
    try:
        price = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
    except NoSuchElementException:
        price = ""

    # Скрејпање на сликите на урлто
    try:
        img = driver.find_element(By.CSS_SELECTOR, "div.woocommerce-product-gallery__image img")
        image_url = img.get_attribute("data-src") or img.get_attribute("src")
    except NoSuchElementException:
        image_url = ""

    # Скрејпање на достапни величини на продуктите
    sizes = []
    try:
        descr = driver.find_element(By.ID, "tab-description")
        driver.execute_script("arguments[0].scrollIntoView(true);", descr)
        div = descr.find_element(By.XPATH, ".//div[contains(text(),'Достапни големини')]")
        ul = div.find_element(By.XPATH, "following-sibling::ul[1]")
        sizes = [li.text.strip() for li in ul.find_elements(By.TAG_NAME, "li") if li.text.strip()]
    except Exception:
        pass

    sizes_str = ", ".join(sizes)

    print(f"[{idx}/{len(product_urls)}] {name} → Sizes: {sizes_str or '—'}")

    data.append({
        "Name": name,
        "Price": price,
        "Sizes": sizes_str,
        "Image_URL": image_url,
        "Product_URL": href
    })

# Зачувување во пандас датафрејм во цсв формат и нивно принтање со име и фостапни величини
df = pd.DataFrame(data)
df.to_csv("kosuli_venti.csv", index=False, encoding="utf-8")

print(f"✔ Scraped {len(df)} products. Saved to kosuli_venti.csv.")
driver.quit()


→ Found 14 products across 2 pages.
[1/14] VE033 SLIM FIT LONG SLEEVE SILVER → Sizes: M, L, XL
[2/14] VE040 SLIM FIT LONG SLEEVE BLUE\WHITE → Sizes: L
[3/14] VE038 SLIM FIT LONG SLEEVE BLUE\LINE → Sizes: L
[4/14] VE036 SLIM FIT LONG SLEEVE WHITE → Sizes: —
[5/14] VE026 SLIM FIT LONG SLEEVE BROWN → Sizes: L
[6/14] VE035 SLIM FIT LONG SLEEVE SILVER → Sizes: M, L
[7/14] VE044 SLIM FIT LONG SLEEVE BLUE → Sizes: —
[8/14] VE025 SLIM FIT LONG SLEEVE BLUE\LINE → Sizes: —
[9/14] VE041 SLIM FIT LONG SLEEVE RED\WHITE → Sizes: M, L, XL, XXL
[10/14] VE034 SLIM FIT LONG SLEEVE BLUE → Sizes: L
[11/14] VE023 SLIM FIT LONG SLEEVE WHITE/BLUE PRINT → Sizes: M, L, XL
[12/14] VE042 SLIM FIT LONG SLEEVE PINK\LINE → Sizes: 42, 44
[13/14] VE039 SLIM FIT LONG SLEEVE BROWN → Sizes: 40, 44
[14/14] VE043 SLIM FIT LONG SLEEVE BLUE\LINE → Sizes: 40, 42, 44
✔ Scraped 14 products. Saved to kosuli_venti.csv.
